In [1]:
#Connect to wandb
#TO DO : how to save models on the weight and bias platform
import wandb
wandb.login()
wandb.init(project="svm", 
           entity="benchmark-nlp",
           name='fake news datasets svm') #CHANGE

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jtonglet (benchmark-nlp). Use `wandb login --relogin` to force relogin


In [2]:
import os
#Move back to the root directory of the project
os.chdir('../../..')

In [3]:
#Load packages
import warnings
import io
import numpy as np
import pandas as pd
from codecarbon import EmissionsTracker
import yaml
from util.dataloader import DataLoader
from preprocessing.preprocessor import Preprocessor
from util.datasplitter import data_splitter
from preprocessing.fasttext_embeddings import FastTextEmbeddings
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, average_precision_score
warnings.filterwarnings("ignore")

In [4]:
#Set constant values
SEED=42
OPT_ITER=10

## Load data

In [5]:
dl = DataLoader(['fake_news'])
data = dl.load()


tweet_preprocessor = Preprocessor(is_tweet=True)
preprocessor = Preprocessor()

#We are not interested in the test sets for hyperparameter optimization

train_gossipcop, val_gossipcop, _ = data_splitter(data['gossipcop'],
                                 preprocessor, 
                                 create_val_set=True,   #No validation set is provided
                                 seed=SEED)

100%|████████████████████████████████████████████████████████████████████████████| 5323/5323 [00:04<00:00, 1096.75it/s]


In [6]:
train_coaid, val_coaid, _ = data_splitter(data['CoAID'],
                                 preprocessor, 
                                 create_val_set=True,   #No validation set is provided
                                          test_split=0.25,
                                          val_split=0.2,
                                 seed=SEED)

5457 rows preprocessed in 6.509600639343262 seconds


In [6]:
train_liar, val_liar, _ = data_splitter(data['liar'],
                                 preprocessor, 
                                 create_val_set=True,   #No validation set is provided
                                 seed=SEED)

10269 rows preprocessed in 5.885899305343628 seconds
1283 rows preprocessed in 0.9678049087524414 seconds
1284 rows preprocessed in 0.7875833511352539 seconds


In [7]:
%%time
#fasttext 
fasttext = FastTextEmbeddings()
fasttext.load_model('fasttext/cc.en.300.bin')

CPU times: total: 18.9 s
Wall time: 42.2 s


In [9]:
embedded_train_gossipcop = fasttext.generate_sentence_embeddings(train_gossipcop['text'])
embedded_val_gossipcop = fasttext.generate_sentence_embeddings(val_gossipcop['text'])
embedded_train_gossipcop['label'] = train_gossipcop['label'].to_list()
embedded_val_gossipcop['label'] = val_gossipcop['label'].to_list()

Starting to generate sentence embeddings


100%|████████████████████████████████████████████████████████████████████████████| 11897/11897 [05:12<00:00, 38.01it/s]


Starting to generate sentence embeddings


100%|██████████████████████████████████████████████████████████████████████████████| 2975/2975 [01:07<00:00, 43.97it/s]


In [8]:
embedded_train_coaid = fasttext.generate_sentence_embeddings(train_coaid['text'])
embedded_val_coaid = fasttext.generate_sentence_embeddings(val_coaid['text'])
embedded_train_coaid['label'] = train_coaid['label'].to_list()
embedded_val_coaid['label'] = val_coaid['label'].to_list()

Starting to generate sentence embeddings


100%|█████████████████████████████████████████████████████████████████████████████| 3273/3273 [00:17<00:00, 185.00it/s]


Starting to generate sentence embeddings


100%|███████████████████████████████████████████████████████████████████████████████| 819/819 [00:04<00:00, 178.04it/s]


In [10]:
embedded_train_liar = fasttext.generate_sentence_embeddings(train_liar['text'])
embedded_val_liar = fasttext.generate_sentence_embeddings(val_liar['text'])
embedded_train_liar['label'] = train_liar['label'].to_list()
embedded_val_liar['label'] = val_liar['label'].to_list()

Starting to generate sentence embeddings


100%|███████████████████████████████████████████████████████████████████████████| 10269/10269 [00:19<00:00, 536.31it/s]


Starting to generate sentence embeddings


100%|█████████████████████████████████████████████████████████████████████████████| 1284/1284 [00:02<00:00, 595.21it/s]


## Hyperopt

In [9]:
import yaml
#Load the template yaml sweep config file for logistic regression
#If the value range for an hyperparameter needs to be changed, better to do it in the .yaml file than in a notebook
with open("config/svm_sweep.yaml", 'r') as stream:
    sweep_config = yaml.safe_load(stream)

In [10]:
#The config is displayed as a nested dictionary
sweep_config

{'method': 'random',
 'entity': 'benchmark-nlp',
 'project': 'hyperopt',
 'metric': {'name': 'loss', 'goal': 'minimize'},
 'parameters': {'C': {'min': 0, 'max': 10, 'distribution': 'uniform'},
  'kernel': {'values': ['linear', 'rbf']},
  'probability': {'value': True},
  'random_state': {'value': 42}}}

#### gossipcop

In [17]:
#Don't forget to name the sweep instance  
name = 'svm_tfidf_gossipcop' #change here
sweep_config['name'] =  name

#Generate a sweep_id
sweep_id = wandb.sweep(sweep_config, project="svm")

def train_tfidf(config = None,
          train=train_gossipcop, #Change here
          val=val_gossipcop): #change here
    '''
    Generic WandB function to conduct hyperparameter optimization with tf-idf vectorizer
    '''
    # Initialize a new wandb run
    with wandb.init(config=config,group=name):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        vec = TfidfVectorizer()
        clf = SVC(C=config.C, 
#                   probability=config.probability,
                  kernel=config.kernel,
                  random_state=config.random_state) #set the hyperparams here
        
        #Create the pipeline
        pipe = Pipeline([('vectorizer',vec),('clf',clf)])
        #Fit the pipeline
        pipe.fit(train['text'],train['label'])
        
        #Make predictions
        pred_val = pipe.predict(val['text'])
#         pred_prob_val = pipe.predict_proba(val['text'])[:,1]
        accuracy = accuracy_score(val['label'],pred_val)
        f1_macro = f1_score(val['label'],pred_val,average='macro')
        aucpc = '-'
        auc = '-'
        #Log metrics on WandB
        wandb.log({"accuracy": accuracy, "f1 macro":f1_macro, "AUC-PC":aucpc, 'AUC':auc })

#Track emissions
tracker = EmissionsTracker(project_name=name,log_level='warning', measure_power_secs=300,
                           output_file='output/emissions_hyperopt.csv')
#Launch the agent
tracker.start()
wandb.agent(sweep_id, train_tfidf,count=OPT_ITER)
tracker.stop()

[codecarbon WARNING @ 13:11:15] No CPU tracking mode found. Falling back on CPU constant mode.


Create sweep with ID: mskvuhxo
Sweep URL: https://wandb.ai/benchmark-nlp/svm/sweeps/mskvuhxo


[codecarbon WARNING @ 13:11:18] We saw that you have a Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz but we don't know it. Please contact us.
wandb: Agent Starting Run: jy564qit with config:
wandb: 	C: 1.5448114798644297
wandb: 	kernel: linear
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.82252
f1 macro,0.75604


wandb: Agent Starting Run: j6zrn35e with config:
wandb: 	C: 0.1057792824586412
wandb: 	kernel: rbf
wandb: 	probability: True
wandb: 	random_state: 42


...

wandb: Agent Starting Run: mdza4j13 with config:
wandb: 	C: 1.783174372431039
wandb: 	kernel: linear
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.81916
f1 macro,0.75355


wandb: Agent Starting Run: weihfsag with config:
wandb: 	C: 3.0250199756773757
wandb: 	kernel: rbf
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.82319
f1 macro,0.75212


0.008253153762995248

In [20]:
#Don't forget to name the sweep instance   
name = 'svm_ft_gossipcop' #CHANGE HERE
sweep_config['name'] = name
#Generate a sweep_id
sweep_id = wandb.sweep(sweep_config, project="svm")

def train_fasttext(config = None,
          train=embedded_train_gossipcop, #CHANGE HERE
          val=embedded_val_gossipcop): #CHANGE HERE
    # Initialize a new wandb run
    with wandb.init(config=config, group=name):
        config = wandb.config
        clf = SVC(C=config.C, 
#                   probability=config.probability,
                  kernel=config.kernel,
                  random_state=config.random_state) #set the hyperparams here
        pipe = Pipeline([('clf',clf)])
        pipe.fit(train.fillna(0).drop(['label'],axis=1),train['label'])
        
        #Make predictions
        pred_val = pipe.predict(val.fillna(0).drop(['label'],axis=1))
#         pred_prob_val = pipe.predict_proba(val.fillna(0).drop(['label'],axis=1))[:,1]
        accuracy = accuracy_score(val['label'],pred_val)
        f1_macro = f1_score(val['label'],pred_val,average='macro')
#         if train['label'].nunique() <=2:
#             aucpc =  average_precision_score(val['label'],pred_prob_val)
#             auc = roc_auc_score(val['label'],pred_prob_val)
#             #Log predictions on WandB
#         else:
        auc = '-'
        aucpc = '-'
        wandb.log({"accuracy": accuracy, "f1 macro":f1_macro, "AUC-PC":aucpc, 'AUC':auc })


#Track emissions
tracker = EmissionsTracker(project_name=name,log_level='warning', measure_power_secs=300,
                           output_file='output/emissions_hyperopt.csv')
#Launch the agent
tracker.start()
wandb.agent(sweep_id, train_fasttext,count=OPT_ITER)
tracker.stop()

[codecarbon WARNING @ 15:11:55] No CPU tracking mode found. Falling back on CPU constant mode.


Create sweep with ID: 498qfn87
Sweep URL: https://wandb.ai/benchmark-nlp/svm/sweeps/498qfn87


[codecarbon WARNING @ 15:11:58] We saw that you have a Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz but we don't know it. Please contact us.
wandb: Agent Starting Run: 0dvs3zum with config:
wandb: 	C: 3.915236013738065
wandb: 	kernel: linear
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.79059
f1 macro,0.67744


wandb: Agent Starting Run: bd57u7td with config:
wandb: 	C: 4.897724640497741
wandb: 	kernel: linear
wandb: 	probability: True
wandb: 	random_state: 42


...

wandb: Agent Starting Run: a16xbq0o with config:
wandb: 	C: 1.1031642164605016
wandb: 	kernel: linear
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.76269
f1 macro,0.60009


wandb: Agent Starting Run: cquqo1ev with config:
wandb: 	C: 8.340172667482582
wandb: 	kernel: rbf
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.80336
f1 macro,0.72418


0.0013233434469840097

####  CoaiD




In [13]:
#Don't forget to name the sweep instance  
name = 'svm_tfidf_coaid' #change here
sweep_config['name'] =  name

#Generate a sweep_id
sweep_id = wandb.sweep(sweep_config, project="svm")

def train_tfidf(config = None,
          train=train_coaid, #Change here
          val=val_coaid): #change here
    '''
    Generic WandB function to conduct hyperparameter optimization with tf-idf vectorizer
    '''
    # Initialize a new wandb run
    with wandb.init(config=config,group=name):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        vec = TfidfVectorizer()
        clf = SVC(C=config.C, 
                  probability=config.probability,
                  kernel=config.kernel,
                  random_state=config.random_state) #set the hyperparams here
        
        #Create the pipeline
        pipe = Pipeline([('vectorizer',vec),('clf',clf)])
        #Fit the pipeline
        pipe.fit(train['text'],train['label'])
        
        #Make predictions
        pred_val = pipe.predict(val['text'])
#         pred_prob_val = pipe.predict_proba(val['text'])[:,1]
        accuracy = accuracy_score(val['label'],pred_val)
        f1_macro = f1_score(val['label'],pred_val,average='macro')
#         if train['label'].nunique() <=2:
#             aucpc =  average_precision_score(val['label'],pred_prob_val)
#             auc = roc_auc_score(val['label'],pred_prob_val)
#         else:
        aucpc = '-'
        auc = '-'
        #Log metrics on WandB
        wandb.log({"accuracy": accuracy, "f1 macro":f1_macro, "AUC-PC":aucpc, 'AUC':auc })

#Track emissions
tracker = EmissionsTracker(project_name=name,log_level='warning', measure_power_secs=300,
                           output_file='output/emissions_hyperopt.csv')
#Launch the agent
tracker.start()
wandb.agent(sweep_id, train_tfidf,count=OPT_ITER)
tracker.stop()

[codecarbon WARNING @ 13:24:18] No CPU tracking mode found. Falling back on CPU constant mode.


Create sweep with ID: 7864icfp
Sweep URL: https://wandb.ai/benchmark-nlp/svm/sweeps/7864icfp


[codecarbon WARNING @ 13:24:23] We saw that you have a Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz but we don't know it. Please contact us.
wandb: Agent Starting Run: xklx30qw with config:
wandb: 	C: 1.2549147050909304
wandb: 	kernel: rbf
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.93529
f1 macro,0.87107


wandb: Agent Starting Run: u31a1sas with config:
wandb: 	C: 9.473743509307642
wandb: 	kernel: rbf
wandb: 	probability: True
wandb: 	random_state: 42


...

wandb: Agent Starting Run: 65s079mp with config:
wandb: 	C: 9.582743813985978
wandb: 	kernel: rbf
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.93895
f1 macro,0.88041


wandb: Agent Starting Run: w8l8q8r9 with config:
wandb: 	C: 9.271378712573156
wandb: 	kernel: linear
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.94505
f1 macro,0.89932


0.0018849464036439843

In [14]:
#Don't forget to name the sweep instance   
name = 'svm_ft_coaid' #CHANGE HERE
sweep_config['name'] = name
#Generate a sweep_id
sweep_id = wandb.sweep(sweep_config, project="svm")

def train_fasttext(config = None,
          train=embedded_train_coaid, #CHANGE HERE
          val=embedded_val_coaid): #CHANGE HERE
    # Initialize a new wandb run
    with wandb.init(config=config, group=name):
        config = wandb.config
        clf = SVC(C=config.C, 
                  probability=config.probability,
                  kernel=config.kernel,
                  random_state=config.random_state) #set the hyperparams here
        pipe = Pipeline([('clf',clf)])
        pipe.fit(train.fillna(0).drop(['label'],axis=1),train['label'])
        
        #Make predictions
        pred_val = pipe.predict(val.fillna(0).drop(['label'],axis=1))
#         pred_prob_val = pipe.predict_proba(val.fillna(0).drop(['label'],axis=1))[:,1]
        accuracy = accuracy_score(val['label'],pred_val)
        f1_macro = f1_score(val['label'],pred_val,average='macro')
#         if train['label'].nunique() <=2:
#             aucpc =  average_precision_score(val['label'],pred_prob_val)
#             auc = roc_auc_score(val['label'],pred_prob_val)
#             #Log predictions on WandB
#         else:
        aucpc = '-'
        auc = '-'
        wandb.log({"accuracy": accuracy, "f1 macro":f1_macro, "AUC-PC":aucpc, 'AUC':auc })


#Track emissions
tracker = EmissionsTracker(project_name=name,log_level='warning', measure_power_secs=300,
                           output_file='output/emissions_hyperopt.csv')
#Launch the agent
tracker.start()
wandb.agent(sweep_id, train_fasttext,count=OPT_ITER)
tracker.stop()

[codecarbon WARNING @ 13:36:59] No CPU tracking mode found. Falling back on CPU constant mode.


Create sweep with ID: exsblwih
Sweep URL: https://wandb.ai/benchmark-nlp/svm/sweeps/exsblwih


[codecarbon WARNING @ 13:37:03] We saw that you have a Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz but we don't know it. Please contact us.
wandb: Agent Starting Run: zymhge0i with config:
wandb: 	C: 5.398691171394323
wandb: 	kernel: linear
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.94017
f1 macro,0.88547


wandb: Agent Starting Run: hb4g82p0 with config:
wandb: 	C: 3.4145513785191883
wandb: 	kernel: rbf
wandb: 	probability: True
wandb: 	random_state: 42


...

wandb: Agent Starting Run: ihdlfqrj with config:
wandb: 	C: 5.182105637836203
wandb: 	kernel: linear
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.94017
f1 macro,0.88547


wandb: Agent Starting Run: 0dnw7l4s with config:
wandb: 	C: 2.8635892835654575
wandb: 	kernel: linear
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.93651
f1 macro,0.87646


0.0017498850723012796

#### liar

In [21]:
#Don't forget to name the sweep instance  
name = 'svm_tfidf_liar' #change here
sweep_config['name'] =  name

#Generate a sweep_id
sweep_id = wandb.sweep(sweep_config, project="svm")

def train_tfidf(config = None,
          train=train_liar, #Change here
          val=val_liar): #change here
    '''
    Generic WandB function to conduct hyperparameter optimization with tf-idf vectorizer
    '''
    # Initialize a new wandb run
    with wandb.init(config=config,group=name):
        config = wandb.config
        vec = TfidfVectorizer()
        clf = SVC(C=config.C, 
#                   probability=config.probability,
                  kernel=config.kernel,
                  random_state=config.random_state) #set the hyperparams here
        
        #Create the pipeline
        pipe = Pipeline([('vectorizer',vec),('clf',clf)])
        #Fit the pipeline
        pipe.fit(train['text'],train['label'])
        
        #Make predictions
        pred_val = pipe.predict(val['text'])
#         pred_prob_val = pipe.predict_proba(val['text'])[:,1]
        accuracy = accuracy_score(val['label'],pred_val)
        f1_macro = f1_score(val['label'],pred_val,average='macro')
#         if train['label'].nunique() <=2:
#             aucpc =  average_precision_score(val['label'],pred_prob_val)
#             auc = roc_auc_score(val['label'],pred_prob_val)
#         else:
        aucpc = '-'
        auc = '-'
        #Log metrics on WandB
        wandb.log({"accuracy": accuracy, "f1 macro":f1_macro, "AUC-PC":aucpc, 'AUC':auc })

#Track emissions
tracker = EmissionsTracker(project_name=name,log_level='warning', measure_power_secs=300,
                           output_file='output/emissions_hyperopt.csv')
#Launch the agent
tracker.start()
wandb.agent(sweep_id, train_tfidf,count=OPT_ITER)
tracker.stop()

[codecarbon WARNING @ 15:20:51] No CPU tracking mode found. Falling back on CPU constant mode.


Create sweep with ID: cd9p0s93
Sweep URL: https://wandb.ai/benchmark-nlp/svm/sweeps/cd9p0s93


[codecarbon WARNING @ 15:20:53] We saw that you have a Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz but we don't know it. Please contact us.
wandb: Agent Starting Run: i33isj0x with config:
wandb: 	C: 2.3737503803591298
wandb: 	kernel: rbf
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.25
f1 macro,0.23714


wandb: Agent Starting Run: gi3c2uvi with config:
wandb: 	C: 2.955685570511525
wandb: 	kernel: linear
wandb: 	probability: True
wandb: 	random_state: 42


...

wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: rocnhtoh with config:
wandb: 	C: 2.51907822832776
wandb: 	kernel: linear
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.23832
f1 macro,0.23438


wandb: Agent Starting Run: hiy8fp7t with config:
wandb: 	C: 0.7100441070863273
wandb: 	kernel: rbf
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.24299
f1 macro,0.18013


0.001354064435838337

In [22]:
#Don't forget to name the sweep instance   
name = 'svm_ft_liar' #CHANGE HERE
sweep_config['name'] = name
#Generate a sweep_id
sweep_id = wandb.sweep(sweep_config, project="svm")

def train_fasttext(config = None,
          train=embedded_train_liar, #CHANGE HERE
          val=embedded_val_liar): #CHANGE HERE
    # Initialize a new wandb run
    with wandb.init(config=config, group=name):
        config = wandb.config
        clf = SVC(C=config.C, 
#                   probability=config.probability,
                  kernel=config.kernel,
                  random_state=config.random_state) #set the hyperparams here
        pipe = Pipeline([('clf',clf)])
        pipe.fit(train.fillna(0).drop(['label'],axis=1),train['label'])
        
        #Make predictions
        pred_val = pipe.predict(val.fillna(0).drop(['label'],axis=1))
#         pred_prob_val = pipe.predict_proba(val.fillna(0).drop(['label'],axis=1))[:,1]
        accuracy = accuracy_score(val['label'],pred_val)
        f1_macro = f1_score(val['label'],pred_val,average='macro')
#         if train['label'].nunique() <=2:
#             aucpc =  average_precision_score(val['label'],pred_prob_val)
#             auc = roc_auc_score(val['label'],pred_prob_val)
#             #Log predictions on WandB
#         else:
        aucpc = '-'
        auc = '-'
        wandb.log({"accuracy": accuracy, "f1 macro":f1_macro, "AUC-PC":aucpc, 'AUC':auc })


#Track emissions
tracker = EmissionsTracker(project_name=name,log_level='warning', measure_power_secs=300,
                           output_file='output/emissions_hyperopt.csv')
#Launch the agent
tracker.start()
wandb.agent(sweep_id, train_fasttext,count=OPT_ITER)
tracker.stop()

[codecarbon WARNING @ 15:29:58] No CPU tracking mode found. Falling back on CPU constant mode.


Create sweep with ID: lpvuntns
Sweep URL: https://wandb.ai/benchmark-nlp/svm/sweeps/lpvuntns


[codecarbon WARNING @ 15:30:01] We saw that you have a Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz but we don't know it. Please contact us.
wandb: Agent Starting Run: mooc1uup with config:
wandb: 	C: 6.670558976421033
wandb: 	kernel: rbf
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.26402
f1 macro,0.25711


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: bi9e471l with config:
wandb: 	C: 0.10324197914680087
wandb: 	kernel: linear
wandb: 	probability: True
wandb: 	random_state: 42


...

wandb: Agent Starting Run: qcfzq1pd with config:
wandb: 	C: 1.3181643038829982
wandb: 	kernel: rbf
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.24065
f1 macro,0.212


wandb: Agent Starting Run: ec7kzsyq with config:
wandb: 	C: 4.409713036244081
wandb: 	kernel: linear
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.22352
f1 macro,0.17715


0.001796613704196457